In [1]:
import numpy as np
import pandas as pd
import datetime as DT

In [2]:
# get the rouped energy data

df = pd.read_csv("Final_Data/nrg_winter1314_hourly_grouped.csv",
                               parse_dates=[0], infer_datetime_format=True)

df.head()

,DateTime,Acorn_grouped,stdorToU,LCLid,energy_sum,KWH/h (per hour)
0,2013-12-01 01:00:00,Adversity,Std,1404,572.822504,0.407993
1,2013-12-01 01:00:00,Adversity,ToU,280,77.949276,0.278390
2,2013-12-01 01:00:00,Affluent,Std,1527,783.538336,0.513123
3,2013-12-01 01:00:00,Affluent,ToU,458,168.465478,0.367829
4,2013-12-01 01:00:00,Comfortable,Std,1113,507.370649,0.455859


In [3]:
len(df.DateTime.unique())

2136

In [8]:
df.rename(columns={"KWH/h (per hour)": "nrg_cons", "LCLid": "N"}, inplace=True)
df.head(20)

,DateTime,Acorn_grouped,stdorToU,N,energy_sum,nrg_cons
0,2013-12-01 01:00:00,Adversity,Std,1404,572.822504,0.407993
1,2013-12-01 01:00:00,Adversity,ToU,280,77.949276,0.278390
2,2013-12-01 01:00:00,Affluent,Std,1527,783.538336,0.513123
3,2013-12-01 01:00:00,Affluent,ToU,458,168.465478,0.367829
4,2013-12-01 01:00:00,Comfortable,Std,1113,507.370649,0.455859
5,2013-12-01 01:00:00,Comfortable,ToU,299,92.464835,0.309247
6,2013-12-01 02:00:00,Adversity,Std,1404,451.117209,0.321309
7,2013-12-01 02:00:00,Adversity,ToU,280,57.067990,0.203814
8,2013-12-01 02:00:00,Affluent,Std,1527,622.320589,0.407545
9,2013-12-01 02:00:00,Affluent,ToU,458,135.295797,0.295406


In [9]:
(df.iloc[3:17,4]*df.iloc[3:17,5]).sum()/df.iloc[3:17,5].sum()

308.43433867107558

In [10]:
# Spliting Date and time

# temp = pd.DatetimeIndex(grouped_nrg_1314['DateTime'])
# grouped_nrg_1314['Date'] = temp.date
# grouped_nrg_1314['Time'] = temp.time
# del grouped_nrg_1314['DateTime']

In [11]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "N"])

# Define a dictionary with the functions to apply for a given column:
f = {'nrg_cons': wm}

# Groupby and aggregate with your dictionary:
df_grouped = df.groupby(["DateTime", "Acorn_grouped"]).agg(f)

df_grouped.head()

#grouped_nrg_1314.groupby('Acorn_grouped').aggregate?


nrg_cons
DateTime            Acorn_grouped          
2013-12-01 01:00:00 Adversity      0.386444
                    Affluent       0.479599
                    Comfortable    0.424813
2013-12-01 02:00:00 Adversity      0.301773
                    Affluent       0.381671

In [12]:
df_grouped.unstack(1).columns[1]

('nrg_cons', 'Affluent')

In [16]:
nrg_1314_regress = df_grouped.unstack(1)
nrg_1314_regress.head()

nrg_cons                      
Acorn_grouped       Adversity  Affluent Comfortable
DateTime                                           
2013-12-01 01:00:00  0.386444  0.479599    0.424813
2013-12-01 02:00:00  0.301773  0.381671    0.314566
2013-12-01 03:00:00  0.253057  0.335496    0.263379
2013-12-01 04:00:00  0.229611  0.312186    0.252521
2013-12-01 05:00:00  0.217060  0.305041    0.253142

In [17]:
nrg_1314_regress.describe()

nrg_cons                          
Acorn_grouped    Adversity     Affluent  Comfortable
count          2136.000000  2136.000000  2136.000000
mean              0.411049     0.645182     0.479236
std               0.127788     0.235429     0.162388
min               0.195915     0.277244     0.210453
25%               0.324687     0.437052     0.370082
50%               0.394685     0.637846     0.467080
75%               0.510501     0.808110     0.593167
max               0.688483     1.191770     0.844111

In [18]:
nrg_1314_regress.to_csv('energy_final_2013_14.csv')